In [24]:
from firedrake import *
import numpy as np
import matplotlib . pyplot as plt
import time


In [25]:
H = 0.004      # Height of the channel, m
L = 0.004      # Width of the channel, m
T_in = Constant(293.15)     # Inlet temperature, K
q_ = 1e5        # Heat flux, W/m^2
rho_f = 997       # Density of the fluid, kg/m^3
mu_f = 8.9e-4     # Dynamic viscosity of the fluid, Pa.s
k_f = 0.6       # Thermal conductivity of the fluid, W/m.K
c_pf = 4182       # Specific heat capacity of the fluid, J/kg.K
k_s = 400         # Thermal conductivity of the solid, W/m.K

#We define inlet velocity to gain a Re of around 4000

Re = 4000
U = Re * mu_f / (rho_f * H)  

print(U)








0.8926780341023068


In [26]:
from firedrake import *
import pyvista as pv
from scipy.interpolate import RegularGridInterpolator
import numpy as np

grid = pv.read("/home/dave/CHT-model/U_x2999_coarse.vtr")
data = grid["U_x_vtk"]
x = np.unique(grid.x)
y = np.unique(grid.y)
z = np.unique(grid.z)
U_field = data.reshape((len(x), len(y), len(z)), order='F')
interp = RegularGridInterpolator((x, y, z), U_field)
Lx_phys = 0.02    # 2 cm
Ly_phys = 0.004   # 4 mm
Lz_phys = 0.004   # 4 mm
nx, ny, nz = grid.dimensions[0]-1, grid.dimensions[1]-1, grid.dimensions[2]-1
mesh = BoxMesh(nx, ny, nz, Lx_phys, Ly_phys, Lz_phys)
V = FunctionSpace(mesh, "CG", 1)
u = Function(V)
coords = mesh.coordinates.dat.data
xmin, xmax, ymin, ymax, zmin, zmax = grid.bounds

# Rescale to physical lengths
coords_scaled = np.zeros_like(coords)
coords_scaled[:, 0] = xmin + coords[:, 0] / Lx_phys * (xmax - xmin)
coords_scaled[:, 1] = ymin + coords[:, 1] / Ly_phys * (ymax - ymin)
coords_scaled[:, 2] = zmin + coords[:, 2] / Lz_phys * (zmax - zmin)

u.dat.data[:] = 1.3e4 * U * interp(coords_scaled)

# Bigger mesh to allocate hot plate
extra_y = 0.001  # 2 mm
ny_extra = 5
Ly_big = Ly_phys + extra_y  # 0.004 + 0.002 = 0.006 m

big_mesh = BoxMesh(nx, ny + ny_extra, nz, Lx_phys, Ly_big, Lz_phys)
S = VectorFunctionSpace(big_mesh, 'CG', 1)

U_LBE = Function(S)
U_LBE.interpolate(as_vector([u, 0*u, 0*u]), allow_missing_dofs=True)

File("CHT-model/u_lbe_scaled.pvd").write(U_LBE)


/home/dave/venv-firedrake/lib/python3.10/site-packages/firedrake/_deprecation.py:65: UserWarning: The use of `File` for output is deprecated, please update your code to use `VTKFile` from `firedrake.output`.
  warn(


In [ ]:
M = FunctionSpace(big_mesh, "DG", 0)

k = Function(M, name="thermal_conductivity")
x, y, z = SpatialCoordinate(big_mesh)
k_expr = conditional(y >= 0.004, k_s, k_f)
k.interpolate(k_expr)
File("CHT-model/k.pvd").write(k)


# Volumetric heat capacity (rho * cp) piecewise
rho_cp = Function(M)
rho_cp_espr = conditional(y < 0.004, rho_f * c_pf, 0.0)  # Assuming solid has very high volumetric heat capacity
rho_cp.interpolate(rho_cp_espr)
File("CHT-model/rho_cp.pvd").write(rho_cp)




/home/dave/venv-firedrake/lib/python3.10/site-packages/firedrake/_deprecation.py:65: UserWarning: The use of `File` for output is deprecated, please update your code to use `VTKFile` from `firedrake.output`.
  warn(
/home/dave/venv-firedrake/lib/python3.10/site-packages/firedrake/_deprecation.py:65: UserWarning: The use of `File` for output is deprecated, please update your code to use `VTKFile` from `firedrake.output`.
  warn(


In [28]:
Z = FunctionSpace(big_mesh, 'CG', 2)
T = TrialFunction(Z)
eta = TestFunction(Z)

bcT_in = DirichletBC(Z, T_in, 1)

def thermal(T, eta, u):
    a = k*inner(grad(T), grad(eta))*dx \
      + (rho_cp) * dot(u, grad(T)) * eta * dx
    L = q_ * eta * ds(4, domain=big_mesh)
    return a, L

def thermal_supg(T, eta, uh):

    ubar = Function(FunctionSpace(big_mesh, 'DG', 0))
    ubar.project(sqrt(inner(uh, uh)))
    h = CellDiameter(big_mesh)
    delta_K = Function(ubar.function_space())
    delta_K = 0.2 * h / (ubar+1e-5)

    a = k * inner(grad(T), grad(eta)) * dx  \
        + (rho_cp) * inner(grad(T), uh) * eta * dx \
        + (delta_K * inner(uh, grad(T)) ) * \
            ((rho_cp) * inner(uh, grad(eta))) * dx
    L = q_ * eta * ds(4, domain=big_mesh)

    return a, L

outfileT = File("CHT-model/output/temperature_3d.pvd")
Th = Function(Z)
# Initial guess is isothermal field at T_in
Th.interpolate(T_in)
Th.rename("Temperature (K)")
outfileT.write(Th)

a_T, L_T = thermal_supg(T, eta, U_LBE)
pb_T = LinearVariationalProblem(a_T, L_T, Th, bcT_in)
solver_T =  LinearVariationalSolver(pb_T)#, solver_parameters=param)
solver_T.solve()
Th.rename("Temperature (K)")
outfileT.write(Th)

